In [10]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

In [4]:
# Define paths to your datasets
train_dir = '/kaggle/input/dataff/dataF/train'
test_dir = '/kaggle/input/dataff/dataF/test'

# Parameters
img_height, img_width = 224, 224
batch_size = 32
epochs = 30

In [5]:
# Data augmentation for the training data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # Set validation split to 20%
)

# Rescaling for the test data
test_datagen = ImageDataGenerator(rescale=1./255)

# Creating the training data generator
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'  # Set as training data
)

# Creating the validation data generator
validation_generator = train_datagen.flow_from_directory(
    train_dir,  # Use same directory as training data
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'  # Set as validation data
)

# Creating the test data generator
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 7116 images belonging to 3 classes.
Found 1778 images belonging to 3 classes.
Found 600 images belonging to 3 classes.


In [6]:
# Get class labels
class_labels = train_generator.classes

# Encode the class labels as integers
label_encoder = LabelEncoder()
integer_encoded_labels = label_encoder.fit_transform(class_labels)
print("Encoded labels:", integer_encoded_labels)

# Calculate class weights
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(integer_encoded_labels),
    y=integer_encoded_labels
)
class_weights = {i: class_weights[i] for i in range(len(class_weights))}
print("Class weights:", class_weights)

Encoded labels: [0 0 0 ... 2 2 2]
Class weights: {0: 0.6753986332574032, 1: 3.2944444444444443, 2: 0.8224687933425797}


In [7]:
def create_cnn_model(input_shape):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(3, activation='softmax'))
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Create and summarize the CNN model
input_shape = (img_height, img_width, 3)
cnn_model = create_cnn_model(input_shape)
cnn_model.summary()

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
cnn_checkpoint = ModelCheckpoint('best_cnn_model.keras', save_best_only=True)

# Train the CNN model
history_cnn = cnn_model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    class_weight=class_weights,
    callbacks=[early_stopping, cnn_checkpoint]
)

/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │    44,302,848 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │         1,539 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 44,397,635 (169.36 MB)

 Trainable params: 44,397,635 (169.36 MB)

...

156/223 ━━━━━━━━━━━━━━━━━━━━ 46s 692ms/step - accuracy: 0.4879 - loss: 1.4325

W0000 00:00:1718716959.685404     137 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


223/223 ━━━━━━━━━━━━━━━━━━━━ 0s 663ms/step - accuracy: 0.5139 - loss: 1.2845

W0000 00:00:1718717002.090565     138 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1718717040.007194     136 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


223/223 ━━━━━━━━━━━━━━━━━━━━ 208s 855ms/step - accuracy: 0.5142 - loss: 1.2828 - val_accuracy: 0.7227 - val_loss: 0.6443
Epoch 2/30
223/223 ━━━━━━━━━━━━━━━━━━━━ 121s 530ms/step - accuracy: 0.6622 - loss: 0.7117 - val_accuracy: 0.7379 - val_loss: 0.5955
Epoch 3/30
223/223 ━━━━━━━━━━━━━━━━━━━━ 119s 523ms/step - accuracy: 0.7289 - loss: 0.6050 - val_accuracy: 0.7452 - val_loss: 0.6204
Epoch 4/30
223/223 ━━━━━━━━━━━━━━━━━━━━ 122s 532ms/step - accuracy: 0.7447 - loss: 0.5568 - val_accuracy: 0.7525 - val_loss: 0.5589
Epoch 5/30
223/223 ━━━━━━━━━━━━━━━━━━━━ 123s 537ms/step - accuracy: 0.7569 - loss: 0.5145 - val_accuracy: 0.8093 - val_loss: 0.4628
Epoch 6/30
223/223 ━━━━━━━━━━━━━━━━━━━━ 125s 544ms/step - accuracy: 0.7882 - loss: 0.4590 - val_accuracy: 0.7987 - val_loss: 0.4782
Epoch 7/30
223/223 ━━━━━━━━━━━━━━━━━━━━ 118s 518ms/step - accuracy: 0.7951 - loss: 0.4505 - val_accuracy: 0.7891 - val_loss: 0.5313
Epoch 8/30
223/223 ━━━━━━━━━━━━━━━━━━━━ 116s 509ms/step - accuracy: 0.7999 - loss: 0.43

In [8]:
# Save the trained CNN model
cnn_model.save('final_cnn_model.keras')

# Evaluate the CNN model
loss_cnn, accuracy_cnn = cnn_model.evaluate(test_generator)
print(f"Test Accuracy (CNN): {accuracy_cnn*100:.2f}%")


19/19 ━━━━━━━━━━━━━━━━━━━━ 8s 456ms/step - accuracy: 0.7254 - loss: 0.6674
Test Accuracy (CNN): 72.17%


W0000 00:00:1718718130.696373     138 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


In [ ]:
# Function to create the MobileNetV2 model
def create_mobilenet_model(input_shape):
    base_model = MobileNetV2(input_shape=input_shape, include_top=False, weights='imagenet')
    base_model.trainable = True

    # Freeze more layers in the base model
    for layer in base_model.layers[:-50]:
        layer.trainable = False

    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dropout(0.5),
        Dense(128, activation='relu', kernel_regularizer=l2(0.001)),
        Dropout(0.5),
        Dense(3, activation='softmax')
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Create and summarize the MobileNetV2 model
mobilenet_model = create_mobilenet_model((img_height, img_width, 3))
mobilenet_model.summary()

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-6)
mobilenet_checkpoint = ModelCheckpoint('best_mobilenet_model.keras', save_best_only=True)

# Train the MobileNetV2 model
history_mobilenet = mobilenet_model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    class_weight=class_weights,
    callbacks=[early_stopping, reduce_lr, mobilenet_checkpoint]
)

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ ?                      │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ ?                      │   0 (unbuilt) │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,257,984 (8.61 MB)

 Trainable params: 1,855,104 (7.08 MB)

 Non-trainable params: 402,880 (1.54 MB)

...

 54/223 ━━━━━━━━━━━━━━━━━━━━ 1:42 606ms/step - accuracy: 0.3984 - loss: 1.9242

W0000 00:00:1718718653.222208     138 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


223/223 ━━━━━━━━━━━━━━━━━━━━ 0s 449ms/step - accuracy: 0.4289 - loss: 1.7849

W0000 00:00:1718718725.035323     137 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1718718749.198863     136 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


223/223 ━━━━━━━━━━━━━━━━━━━━ 155s 579ms/step - accuracy: 0.4291 - loss: 1.7840 - val_accuracy: 0.6159 - val_loss: 1.0611 - learning_rate: 1.0000e-05
Epoch 2/30
223/223 ━━━━━━━━━━━━━━━━━━━━ 117s 512ms/step - accuracy: 0.5741 - loss: 1.1681 - val_accuracy: 0.6625 - val_loss: 1.0159 - learning_rate: 1.0000e-05
Epoch 3/30
223/223 ━━━━━━━━━━━━━━━━━━━━ 116s 506ms/step - accuracy: 0.6835 - loss: 0.9203 - val_accuracy: 0.6935 - val_loss: 0.9772 - learning_rate: 1.0000e-05
Epoch 4/30
223/223 ━━━━━━━━━━━━━━━━━━━━ 116s 508ms/step - accuracy: 0.7734 - loss: 0.7067 - val_accuracy: 0.7238 - val_loss: 0.9442 - learning_rate: 1.0000e-05
Epoch 5/30
223/223 ━━━━━━━━━━━━━━━━━━━━ 118s 515ms/step - accuracy: 0.8064 - loss: 0.6497 - val_accuracy: 0.7492 - val_loss: 0.8901 - learning_rate: 1.0000e-05
Epoch 6/30


In [ ]:

# Save the trained MobileNetV2 model
mobilenet_model.save('final_mobilenet_model.keras')

# Evaluate the MobileNetV2 model
loss_mobilenet, accuracy_mobilenet = mobilenet_model.evaluate(test_generator)
print(f"Test Accuracy (MobileNetV2): {accuracy_mobilenet*100:.2f}%")

In [ ]:
# Plot training history
def plot_training_history(history, title='Model'):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(1, len(acc) + 1)
    
    plt.figure(figsize=(12, 6))
    
    plt.subplot(1, 2, 1)
    plt.plot(epochs, acc, 'bo-', label='Training accuracy')
    plt.plot(epochs, val_acc, 'ro-', label='Validation accuracy')
    plt.title(f'{title} Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(epochs, loss, 'bo-', label='Training loss')
    plt.plot(epochs, val_loss, 'ro-', label='Validation loss')
    plt.title(f'{title} Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.show()

# Plot training history for the CNN model
plot_training_history(history_cnn, title='CNN Model')

# Plot training history for the MobileNetV2 model
plot_training_history(history_mobilenet, title='MobileNetV2 Model')



In [ ]:
# Function to load and preprocess a single image
def load_and_preprocess_image(img_path, target_size=(224, 224)):
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=target_size)
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Normalize to [0, 1] range
    return img_array

# Function to predict and display the result
def predict_image_class(model, img_path, class_indices):
    img_array = load_and_preprocess_image(img_path)
    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction, axis=1)
    class_labels = {v: k for k, v in class_indices.items()}
    predicted_label = class_labels[predicted_class[0]]
    
    # Display the image
    img = tf.keras.preprocessing.image.load_img(img_path)
    plt.imshow(img)
    plt.title(f"Predicted: {predicted_label}")
    plt.axis('off')
    plt.show()

# Load the class indices from the training generator
class_indices = train_generator.class_indices

# Test with new images
img_paths = [
    '/kaggle/input/testing/acc.jpg',
    '/kaggle/input/testing/fireAcc.jpg'
]

for img_path in img_paths:
    print(f"Predicting for image: {img_path}")
    print("CNN Model Prediction:")
    predict_image_class(cnn_model, img_path, class_indices)
    print("MobileNet Model Prediction:")
    predict_image_class(mobilenet_model, img_path, class_indices)